In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE
import statsmodels.api as sm

In [2]:
loan=pd.read_csv('../.Database/merged_cleaned_loan.csv')

In [3]:
loan['home_ownership'].value_counts()

home_ownership
MORTGAGE    126680
RENT        109239
OWN          21638
OTHER          176
NONE            43
ANY              1
Name: count, dtype: int64

In [4]:
# As there is only 1 observation with home_ownership = ANY,
# we will drop that observation to avoid issues with regression
loan = loan.drop(loan[loan['home_ownership'] == 'ANY'].index)

In [5]:
# Drop non-numerical columns from X
X=loan.drop(columns=['addr_state','emp_title','loan_status','issue_d','issue_y','issue_m','title'])

# Drop annual_inc, to use log_annual_inc
X=X.drop(columns=['annual_inc'])

# Drop outcome variables from X
X=X.drop(columns=['loan_status_grouped','grade','sub_grade', 'int_rate'])

# Select outcome variable for Y
y=loan['loan_status_grouped']

In [6]:
#Separate numerical and categorical columns
categorical_columns = ['home_ownership', 'term','purpose','emp_length']
numerical_columns = X.select_dtypes(include=['float64']).columns

# Apply OneHotEncoding to categorical columns before splitting
# dtype=int to get binary (0 or 1) values
X_encoded = pd.get_dummies(X, columns=categorical_columns, dtype=int)

# Drop the following dummies to avoid dummy variable trap
# This ensures K-1 dummies for K groups
# Dropped dummies represent the base group
# The coefficients of the regressed dummies will be relative to the base group
X_encoded = X_encoded.drop(columns=['home_ownership_MORTGAGE','term_ 36 months','purpose_car','emp_length_0-1 years'])

# Now perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Ensure the train and test sets have the same structure (dummies already handled)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Apply scaling to numerical columns
scaler = StandardScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

In [7]:
# Drop rows with NaN values
X_train = X_train.dropna()
X_test = X_test.dropna()

# Ensure y_train matches X_train, y_test matches X_test
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

In [9]:
# Checking for higher collinear predictors
vif_data = pd.DataFrame()
vif_data["feature"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
print(vif_data)

                         feature       VIF
0                    delinq_2yrs  1.042407
1                            dti  1.247989
2                 inq_last_6mths  1.050614
3                      loan_amnt  1.690072
4         mths_since_last_delinq  1.033256
5                        pub_rec  1.033332
6                      total_acc  1.359023
7                 acc_now_delinq  1.015170
8                 log_annual_inc  1.687369
9   Effective Federal Funds Rate  1.057405
10            state_unemployment  1.093257
11           home_ownership_NONE  1.001198
12          home_ownership_OTHER  1.003501
13            home_ownership_OWN  1.189636
14           home_ownership_RENT  2.168830
15               term_ 60 months  1.579519
16           purpose_credit_card  2.343303
17    purpose_debt_consolidation  5.209311
18           purpose_educational  1.021347
19      purpose_home_improvement  1.396010
20                 purpose_house  1.053488
21        purpose_major_purchase  1.180971
22         

In [10]:
# The data has about 5 times as many repaid loans (0) as defaulted loans (1)
# Such class balance issues can skew predictions greatly
# We can use SMOTE (Synthetic Minority Over-sampling Technique) to oversample the minority class
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [11]:
# Using statsmodels library to run Logit Regression

# Add a constant to the independent variables
X_train_with_const = sm.add_constant(X_train)
X_test_with_const = sm.add_constant(X_test)

# Fit the logistic regression model
logit_model = sm.Logit(y_train, X_train_with_const)

# Fit the model and obtain the result object
result = logit_model.fit(maxiter=1000)

Optimization terminated successfully.
         Current function value: 0.626021
         Iterations 6


In [12]:
# Print the summary of the logistic regression results
print(result.summary())

                            Logit Regression Results                           
Dep. Variable:     loan_status_grouped   No. Observations:               323738
Model:                           Logit   Df Residuals:                   323705
Method:                            MLE   Df Model:                           32
Date:                 Wed, 30 Oct 2024   Pseudo R-squ.:                 0.09684
Time:                         23:18:39   Log-Likelihood:            -2.0267e+05
converged:                        True   LL-Null:                   -2.2440e+05
Covariance Type:             nonrobust   LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            2.0782      0.024     85.378      0.000       2.031       2.126
delinq_2yrs                      0.0947      0.004     24.184      0.

In [13]:
# Print odds ratios in descending order
odds_ratios = np.exp(result.params)
odds_ratios = pd.Series(odds_ratios)
odds_ratios = odds_ratios.sort_values(ascending=False)
print('\nOdds Ratios\n',odds_ratios)

# Odds ratios are predicted probabilities of the effect of X variables
# odds_ratio > 1 : variable increases the probability of default (by odds_ratio - 1)
# odds_ratio < 1 : variable decreases the probability of default (by 1 - odds_ratio)
# odds_ratio = 1 : variable has no effect on the probability of default


Odds Ratios
 const                           7.990251
term_ 60 months                 1.741865
dti                             1.363149
loan_amnt                       1.343167
inq_last_6mths                  1.154451
delinq_2yrs                     1.099380
mths_since_last_delinq          1.033328
pub_rec                         1.030115
home_ownership_RENT             1.016499
acc_now_delinq                  1.003895
Effective Federal Funds Rate    0.955707
state_unemployment              0.887938
total_acc                       0.813764
log_annual_inc                  0.722447
emp_length_6-9 years            0.687386
emp_length_10+ years            0.685114
emp_length_2-5 years            0.678367
home_ownership_OWN              0.622077
home_ownership_OTHER            0.348216
purpose_small_business          0.182366
purpose_other                   0.147107
home_ownership_NONE             0.142560
purpose_debt_consolidation      0.136690
purpose_home_improvement        0.112253
pu